In [1]:
from typing import Tuple, List, Dict
from dataclasses import dataclass

triples: List[Tuple[str, str, str]] = []

create_match = lambda tr: lambda s, p, o: [t for t in tr if (not s or t[0] == s) and (not p or t[1] == p) and (not o or t[2] == o)]
match = lambda s = None, p = None, o = None: create_match(triples)(s, p, o)
match_all = lambda s = None, p = None, o = None : list(match(s, p, o))
match_first = lambda s = None, p = None, o = None : next(iter(match(s, p, o) or []), (None, None, None))


In [2]:
import shlex 
    
filenames = ['emmo.nt']
for filename in filenames:
    with open(filename, 'r') as file:            
        lines = shlex.split(file.read());        
        for i in range(0, len(lines), 4):
            triples.append((lines[i], lines[i+1], lines[i+2])) 
          

In [7]:
from ipytree import Tree, Node
from ipywidgets import HBox, HTML, Output, Layout
import html
from namespaces import RDFS, SKOS, EMMO

a = Output(layout=Layout(min_width='50%'))
o = Output()
nodemap = {}

style = """
<!-- CSS only -->
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous">
<style type='text/css'>
        p {
            border: 1px solid rgb(200, 200, 200);
            background: rgb(234, 234, 234);
            padding: 5px;
            margin: 5px;
        }
        p.note {
            background: #ffffcc;
            border: 1px solid gold;
        }
        </style>
"""

def handle_click(event):    
    concept = nodemap[event['owner']._id]
    a.clear_output()
    with a:
        (_,_,desc) = match_first(concept, EMMO.elucidation, None)
        comments = match_all(concept, RDFS.comment, None)
        comments = "\n".join(list([html.unescape(c[2]).replace('@en','\n') for c in comments])).replace('\\n','<br>')
        desc = html.unescape(desc).replace('@en','') if desc else ""        
        display(HTML(value=f"""{style}
        <p>
          <center><h2>{event['owner'].name}</h2></center>
        </p>
        <p>
          {html.escape(concept)}
        </p>
        <p class='note'>
          {desc}<br>
        </p>
        <p class='note'>
          {comments}
        </p>
        """))

def gen_tree(root):
    l = []
    
    # Recursively generate and append sub-nodes
    for (s,_,_) in match(None, RDFS.subClassOf, root):        
        l.append(gen_tree(s))
        
    # Find the prefLabel (node name)
    (_,_,lbl) = match_first(root, SKOS.prefLabel, None)
    lbl = lbl.replace('@en','')
    if l:
        node = Node(lbl, l, opened=False, icon_style="success")
    else:
        node = Node(lbl, l, opened=False, icon="leaf", icon_style="success")
        
    # Add observer to handle interactions
    node.observe(handle_click, 'selected')
    nodemap[node._id] = root
    return node

emmo_node = gen_tree('<http://emmo.info/emmo#EMMO_802d3e92_8770_4f98_a289_ccaaab7fdddf>')

In [9]:
tree=Tree(layout=Layout(min_width='50%'))

tree.add_node(emmo_node)
box = HBox(children=[tree, a], layout=Layout(align_items='stretch', width='100%'))

display (box)
#display (o)
